In [42]:
(pl.scan_parquet("../../parquets/allplayers.parquet"
                )
                
    .filter(pl.col("id")=="freef001")
                ).collect()

id,last,first,bat,throw,team,g,g_p,g_sp,g_rp,g_c,g_1b,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr,first_g,last_g,season
str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""freef001""","""Freeman""","""Freddie""","""L""","""R""","""ATL""",20,0,0,0,0,12,0,0,0,0,0,0,0,0,11,2,20100901,20101002,2010
"""freef001""","""Freeman""","""Freddie""","""L""","""R""","""ATL""",157,0,0,0,0,156,0,0,0,0,0,0,0,0,1,0,20110331,20110928,2011
"""freef001""","""Freeman""","""Freddie""","""L""","""R""","""ATL""",148,0,0,0,0,147,0,0,0,0,0,0,0,0,2,0,20120405,20121005,2012
"""freef001""","""Freeman""","""Freddie""","""L""","""R""","""ATL""",151,0,0,0,0,151,0,0,0,0,0,0,0,0,0,1,20130401,20131007,2013
"""freef001""","""Freeman""","""Freddie""","""L""","""R""","""NLS""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""freef001""","""Freeman""","""Freddie""","""L""","""R""","""NLS""",1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,20220719,20220719,2022
"""freef001""","""Freeman""","""Freddie""","""L""","""R""","""LAN""",164,0,0,0,0,164,0,0,0,0,0,0,0,0,0,0,20230330,20231011,2023
"""freef001""","""Freeman""","""Freddie""","""L""","""R""","""NLS""",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20230711,20230711,2023


In [46]:
players = (
    pl.scan_parquet("../../parquets/allplayers.parquet")
    .filter(pl.col("g")>30)
    .with_columns(
        (pl.col("first") + pl.lit(" ") + pl.col("last")).alias("name"),
        pl.count("season").over("id").alias("count"),
    )
    .filter(pl.col("name").is_not_null())
    .filter(pl.col("count") > 5)
    .select(
        pl.col("name"),
        pl.col("id"),
    )
    .unique()
    .sort("name")
).collect()
players

name,id
str,str
"""A. J. Minter""","""minta001"""
"""A.J. Burnett""","""burna001"""
"""A.J. Ellis""","""ellia001"""
"""A.J. Minter""","""minta001"""
"""A.J. Pierzynski""","""piera001"""
…,…
"""Zack Taylor""","""taylz101"""
"""Zack Wheat""","""wheaz101"""
"""Zack Wheeler""","""wheez001"""


In [28]:
import polars as pl
players = (
    pl.scan_parquet("../../parquets/allplayers.parquet")
    .with_columns(
        (pl.col("first") + pl.lit(" ") + pl.col("last")).alias("name"),
        pl.count("season").over("id").alias("count"),
    )
    .filter(pl.col("count") > 5)
    .select(
        pl.col("name"),
        pl.col("id"),
        )
)
players.collect()

name,id
str,str
"""John Anderson""","""andej101"""
"""Jimmy Barrett""","""barrj103"""
"""Shad Barry""","""barrs101"""
"""Shad Barry""","""barrs101"""
"""Harry Bay""","""bay-h101"""
…,…
"""Christian Yelich""","""yelic001"""
"""Alex Young""","""youna002"""
"""Alex Young""","""youna002"""


In [32]:
import polars as pl
batting = (
    pl.scan_parquet("../../parquets/batting.parquet")
    .with_columns(
        pl.col("date").cast(pl.String)
        .str.strptime(pl.Date, "%Y%m%d")
        .dt.year()
        .alias("year")
    )
    .group_by(
        "id", "year"
    )
    .agg(pl.col("b_h").sum(), pl.col("b_ab").sum())
    .with_columns((pl.col("b_h") / pl.col("b_ab")).alias("avg"))
    .filter(pl.col("b_ab") > 100)
    .with_columns(
        pl.count("year").over("id").alias("count")
    )
    .filter(pl.col("id")=="freef001")
    .select(
        pl.col("id").alias("unique_id"),
        pl.col("year").alias("ds"),
        pl.col("avg").alias("y"),
    )
).collect()
batting

unique_id,ds,y
str,i32,f64
"""freef001""",2014,0.289474
"""freef001""",2017,0.306818
"""freef001""",2024,0.278992
"""freef001""",2012,0.262868
"""freef001""",2013,0.319224
…,…,…
"""freef001""",2019,0.291262
"""freef001""",2020,0.330739
"""freef001""",2021,0.300912


In [30]:
import statsforecast.models
models = [
    statsforecast.models.AutoARIMA(),
    # statsforecast.models.AutoETS(),
    # statsforecast.models.AutoRegressive(10),
    # statsforecast.models.HoltWinters(),
    # statsforecast.models.HistoricAverage(),
]

In [33]:
# Instantiate StatsForecast class as sf
sf = statsforecast.StatsForecast( 
    models=models,
    freq=1, 
    n_jobs=-1,
    verbose=True,
)

In [37]:
forecasts_df = sf.forecast(df=batting, h=1, level=[75])
forecasts_df.head()

Forecast:   0%|          | 0/1 [00:00<?, ?it/s]

Forecast: 100%|██████████| 1/1 [00:00<00:00, 12.61it/s]


unique_id,ds,AutoARIMA,AutoARIMA-lo-75,AutoARIMA-hi-75
str,i64,f64,f64,f64
"""freef001""",2025,0.300002,0.275959,0.324044


In [8]:
forecasts_df

unique_id,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
str,i64,f64,f64,f64
"""freef001""",2025,0.300002,0.265624,0.33438


In [9]:
results = (
    forecasts_df
    .join(pl.read_parquet("../../parquets/allplayers.parquet").unique("id"), left_on="unique_id", right_on="id", how="left")
    .with_columns((pl.col("first") + pl.lit(" ") + pl.col("last")).alias("name"))
    .select(
        "name",
        "unique_id",
        "ds",
        "AutoARIMA",
        "AutoARIMA-lo-90",
        "AutoARIMA-hi-90",
    )
    )
results.write_parquet("arima_forecasts.parquet")
results

name,unique_id,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
str,str,i64,f64,f64,f64
"""Freddie Freeman""","""freef001""",2025,0.300002,0.265624,0.33438
